In [1]:
import pandas as pd
import requests
import geopandas as gpd
import dask.dataframe as dd
import os
from dbfread import DBF
from tqdm import tqdm
import fiona
from fiona.crs import from_epsg
from tqdm import tqdm

In [2]:
SHP_DIR = '../local/ptv-datashare-orders/PTV'
SHP_GDFK_PTV : gpd.GeoDataFrame = { f.split('.')[0]: gpd.read_file(os.path.join(SHP_DIR, f)) for f in os.listdir(SHP_DIR) if f.endswith('.shp') }
for f in os.listdir(SHP_DIR):
    if f.endswith('.txt'):
        gdf_name = f.removesuffix('_column_names.txt').upper()
        with open(os.path.join(SHP_DIR, f), 'r') as file:
            gdf_column_names = [line.strip() for line in file.readlines()][4:]
        assert gdf_name in SHP_GDFK_PTV, f'{gdf_name} not in GDFS'
        for line in gdf_column_names:
            assert ' = ' in line, f'Invalid line: {line}'
        gdf_column_names = { line.split(' = ')[0]: line.split(' = ')[1] for line in gdf_column_names }
        SHP_GDFK_PTV[gdf_name].rename(columns=gdf_column_names, inplace=True)

In [3]:
f = '../local/ptv-datashare-orders/VMTRANS/tr_road_column_names.txt'
gdf_name = f.removesuffix('_column_names.txt').upper()
with open(f) as file:
    gdf_column_names = [line.strip() for line in file.readlines()][4:]
for line in gdf_column_names:
    assert ' = ' in line, f'Invalid line: {line}'
gdf_column_names = { line.split(' = ')[0]: line.split(' = ')[1] for line in gdf_column_names }
# SHP_GDFK_PTV[gdf_name].rename(columns=gdf_column_names, inplace=True)

In [4]:
# SHP_VICROADS_FILES = [(int(f.split('.')[0].split('-')[1]), os.path.join('../local/ptv-datashare-orders/VMTRANS_split/', f)) for f in os.listdir('../local/ptv-datashare-orders/VMTRANS_split/') if f.endswith('.shp')]
# SHP_VICROADS_FILES = sorted(SHP_VICROADS_FILES, key=lambda x: x[0])
# SHP_VICROADS_FILES = SHP_VICROADS_FILES[:-1]
# SHP_GDFK_VICROADS = {}

# with tqdm(total=len(SHP_VICROADS_FILES)) as pbar:
#     for gdf_name, filepath in SHP_VICROADS_FILES:
#         SHP_GDFK_VICROADS[gdf_name] = gpd.read_file(filepath)
#         pbar.update(1)
# 350m - 6 hours

In [5]:
# GDF_VICROADS : gpd.GeoDataFrame = gpd.GeoDataFrame(pd.concat(SHP_GDFK_VICROADS.values(), ignore_index=True))
# GDF_VICROADS.rename(columns=gdf_column_names, inplace=True)

In [6]:
SHP_MELROADS_FILES = [(int(f.split('.')[0].split('-')[1]), os.path.join('../local/ptv-datashare-orders/VMTRANS_MEL_split/', f)) for f in os.listdir('../local/ptv-datashare-orders/VMTRANS_MEL_split/') if f.endswith('.shp')]
SHP_MELROADS_FILES = sorted(SHP_MELROADS_FILES, key=lambda x: x[0])
SHP_MELROADS_FILES = SHP_MELROADS_FILES[:-1]
SHP_GDFK_MELROADS = {}

with tqdm(total=len(SHP_MELROADS_FILES)) as pbar:
    for gdf_name, filepath in SHP_MELROADS_FILES:
        SHP_GDFK_MELROADS[gdf_name] = gpd.read_file(filepath)
        pbar.update(1)
# 5m - 10m
        
GDF_MELROADS : gpd.GeoDataFrame = gpd.GeoDataFrame(pd.concat(SHP_GDFK_MELROADS.values(), ignore_index=True))
# 2s - 5s

100%|██████████| 500/500 [04:17<00:00,  1.94it/s]
C:\Users\An\AppData\Local\Temp\ipykernel_23132\321312405.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  GDF_MELROADS : gpd.GeoDataFrame = gpd.GeoDataFrame(pd.concat(SHP_GDFK_MELROADS.values(), ignore_index=True))


In [7]:
GDF_MELROADS.rename(columns=gdf_column_names, inplace=True)

for col in GDF_MELROADS.columns:
    if GDF_MELROADS[col].isna().all():
        GDF_MELROADS.drop(columns=[col], inplace=True)
        # 7s
GDF_MELROADS['length'] = GDF_MELROADS.geometry.to_crs(epsg=3857).length
GDF_MELROADS['line'] = GDF_MELROADS.geometry.apply(lambda x: x.coords)
GDF_MELROADS['line_start'] = GDF_MELROADS['line'].apply(lambda x: x[0])
GDF_MELROADS['line_end'] = GDF_MELROADS['line'].apply(lambda x: x[-1])
assert GDF_MELROADS.groupby(['FROM_UFI'])['line_start'].nunique().unique() == [1]
assert GDF_MELROADS.groupby(['TO_UFI'])['line_end'].nunique().unique() == [1]

In [ ]:
GDF_MELROADS.columns

In [ ]:
GDF_MELROADS[GDF_MELROADS['LEFT_LOCALITY'] != GDF_MELROADS['RIGHT_LOCALITY']][['EZI_ROAD_NAME', 'LEFT_LOCALITY', 'RIGHT_LOCALITY', 'geometry']]

In [ ]:
# shp2pgsql -s 7844 -I TR_ROAD.shp tr_road | psql  -d gis_db -U postgres >> log.txt 2>&1

In [9]:
gdf = gpd.read_file('../local/ptv-spatial-datasets/PTV_METRO_TRAIN_STATION.shp')

In [13]:
gdf[gdf['STOP_NAME'].str.contains("Melbourne Central")][['LATITUDE', 'LONGITUDE']].iloc[0]

LATITUDE     -37.809939
LONGITUDE    144.962594
Name: 92, dtype: float64